In [ ]:
#First of all I have to acess my drive in order to retrieve the trento pbf osm or xml file
import shutil
import os

In [ ]:
!pip install osmium
!pip install --no-binary :all: osmium
!pip install PyYAML==5.4.1

# **Funzionamento del file .yaml**

Parte riguardante i campi "way-keys":</br>
- Campo way-keys è costituito da vari campi key, il campo key è costituito dagli elementi name, values, values_not_to_have, tags, questions, must_have_tag, must_not_have_tag.
> - name: Nome della chiave che andremo a cercare (per es. highway)
> - values: costituito da diversi campi value, rappresentano i valori che sono associati alla chiave specificata dal campo name. (per es. motorway). Questo valore dovrà essere quindi presente nella chiave
> - values_not_to_have: costituito da diversi campim value, rappresentano i valori che non possono essere associati alla chiave specificata dal campo name. (per es. motorway). Se la chiave ha questo valore allora verrà scartata
> - tags: costituito da vari campi tag
>> - tag: costituito dai campi name, value, question, score, validating e answers. 
>>> - name: il campo name all'interno del tag rappresenta il tag che abbiamo scelto. Cerchiamo quindi se questo tag è presente nell'elemento selezionato. (per es. bicycle)
>>> - value: rappresenta il valore del tag. (per es. "yes") 
>>> - question: rappresente la domanda da effettuare nel caso l'elemento scelto abbia come tag quello indicato con valore indicato.
>>> - score: rappresenta il punteggio che viene assegnato al giocatore se risponde.
>>> - validating: campo che mostra se la domanda sarà di validazione oppure no.
>>> - answers: campo che indica le possibili risposte che lo user può dare alla domanda. Se il campo è none allora è una missione di validazione e le uniche risposte possibili sono sì o no.

> - questions: costituito da vari campi question, rappresentano le domande da effettuare per qualunque elemento avente come chiave quella richiesta. 
>> - question: costituito da campi question, score, tagAnswer e answers.
>>> - question: rappresenta la domanda associata
>>> - score: rappresenta il punteggio associato se lo user risponde alla domanda.
>>> - tagAnswer: rappresenta il tag associato alla risposta.
>>> - answers: rappresenta le risposte che lo user può inserire.

> - must_have_tag: Costituito da vari campi tag, rappresenta tutti i tag che la nostra chiave deve avere. Contiene inoltre i valori che il tag può o non può avere
>> - tag: costituito da campi name, values_to_discard, values_to_have
>>> - name: nome del tag (per es. bicycle)
>>> - values_to_discard: i valori associati al campo tag che se posseduti fanno scartare l'elemento (per es. "yes")
>>> - values_to_have: i valori associati al campo tag che devono essere posseduti affinchè l'elemento non venga scartato (per es. "yes")
> - must_not_have_tag: costituito da vari campi tag, rappresentano i nomi dei tag che se presenti nel nostro elemento lo fanno scartare.

Parte riguardante i campo "node-keys":</br>
- Il campo node-keys è costituito da vari campi key, il campo key è costituito dagli elementi name, values, tags, questions.
> - name: nome della chiave che ricerchiamo (per es. amenity)
> - values: costituito da vari campi value
>> - value: campi aventi un elemento name e uno question
>>> - name: costituisce il valore che deve possedere la chaive ricercata (per es. bicycle_parking)
>>> - question: La domanda associata nel caso in cui l'elemento fosse presente
> - tags: costituito da vari campi tag, rappresentano i tag che l'elemento deve possedere oltre alla chiave
>> - tag: costituito dai campi name, value e question
>>> - name: il nome del tag ricercato (per es. bicycle)
>>> - value: il valore che deve possedere il tag (per es. yes)
>>> - question: La domanda associata se il tag è presente
> - questions: costituito da vari campi question che indicano le domande da effettuare se la chiave viene trovata. Questo campo viene visualizzato indipendentemente dal valore della chiave. 

In [ ]:
import yaml
from yaml.loader import SafeLoader

yamlFile="tmp/tagAndKeys_NewVersion_Answer.yaml"
trentoFile="TrentoFileXml/Trento.xml"
cityDirectory = "CityDirectory"
cityFiles = [] #just to cycle
cityFilesOrder = [] #useful to know which data belong to which city and keep them separated.

cityD = os.listdir(cityDirectory)
for i in cityD:
    cityFiles.append(cityDirectory + "/" + i)
    cityFilesOrder.append(i.replace('.xml',''))

print(cityFiles)

#EXAMPLE ON HOW TO USE YAML FILES
# Open the file and load the file
#once Python exits from the “with” block, the file is automatically closed.
with open(yamlFile) as f:
    data = yaml.load(f, Loader=SafeLoader)
    sorted_data = yaml.dump(data, sort_keys=True)

Questa parte di Osmium in realtà penso si possa eliminare, TODO PROVARE A ELIMINARE QUESTA SEZIONE E MODIFICARE CODICE IN MODO CHE RIFUNZIONI. MI BASTA ESTRARRE OGNI VIA E NODO ANZICHE' QUELLI CHE HANNO A CHE FARE SOLO CON LE BICICLETTE

In [ ]:
import osmium
!pip install --no-binary :all: osmium

#DEVO METTERE HIGHWAY != DA MOTORWAY, MOTORWAY_LINK, motorway_junction, 

class BicycleHandler(osmium.SimpleHandler):
    def __init__(self):
        super(BicycleHandler, self).__init__()
        self.ways = []
        self.nodes = []
        nodeKeyName = []
        wayKeyName = []

    
    def node(self, n):
        tags = {}
        if len(n.tags) > 0:
            for tag in n.tags:
                tags[tag.k] = tag.v

        _node = {
                  'id':        n.id,
                  'version':   n.version,
                  'uid':       n.uid,
                  'timestamp': n.timestamp,
                  'changeset': n.changeset,
                  'tags':      tags,
                  'lat':       n.location.lat,
                  'lon':       n.location.lon,
                }
        for k in self.nodeKeyName:
          if n.tags.get(k) != "no" and n.tags.get(k) is not None:
            self.nodes.append(_node)


    def way(self, w):
      tags = {}
      if len(w.tags) > 0:
            for tag in w.tags:
                tags[tag.k] = tag.v

      _way = {
                  'id':        w.id,
                  'version':   w.version,
                  'uid':       w.uid,
                  'timestamp': w.timestamp,
                  'changeset': w.changeset,
                  'tags':      tags,
                }
      
      for k in self.wayKeyName:
        if w.tags.get(k) != "no" and w.tags.get(k) is not None:
          self.ways.append(_way)

outWayKeyNames = []
outNodeKeyNames = []

def elementAlreadyHere(name_way, my_list):
  for i in my_list:
    if i == name_way:
      return True
  return False

with open(yamlFile) as f:
    data = yaml.load(f, Loader=SafeLoader)

    for keys,value in data['way-keys'].items():
        print(data["way-keys"][keys]["name"])
        #print("PRINTING KEYS AND VALUES")
        #print(keys)
        #print(value)
        print("FINISH PRINTING KEYS AND VALUES")
        if data["way-keys"][keys]["name"]!="None":
          #Prima verifico se è già presente
          if elementAlreadyHere(data["way-keys"][keys]["name"], outWayKeyNames) == False:
            print("WEEE")
            outWayKeyNames.append(data["way-keys"][keys]["name"])
          #outNodeKeyNames.append(data["node-keys"][keys]["name"])
    for keys,value in data['node-keys'].items():
        print(data["node-keys"][keys]["name"])
        if data['node-keys'][keys]['name']is not None:
          #Prima verifico se è già presente
          if elementAlreadyHere(data["node-keys"][keys]["name"], outNodeKeyNames) == False:
            outNodeKeyNames.append(data["node-keys"][keys]["name"])

allWayKeyNames = []
allNodeKeyNames = []

for i in cityFiles:
  b = BicycleHandler()
  b.wayKeyName = outWayKeyNames
  b.nodeKeyName = outNodeKeyNames
  b.apply_file(i)
  #print("WAAAAAAA")
  #print(b.ways[0])
  allWayKeyNames.append(b.ways)
  allNodeKeyNames.append(b.nodes)
  print(len(b.ways))
  print(len(b.nodes))
print(len(allWayKeyNames))
print(len(allNodeKeyNames))
#Come faccio a capire quali sono di quale città?

In [ ]:
print(allWayKeyNames[0][1]) #b.ways[1]

In [ ]:
#Initialize the csv file with the elements in the list
def createFile(filename, myList):
  with open(filename, 'w', newline='') as file:
    writer = csv.writer(file, delimiter='|')
    writer.writerows(myList)
    file.close()

In [ ]:
#add an element to the list
def initList(myList, myString):
  myList.append(myString)
  return myList

#Initialize the csv file with the elements in the list
def createFile(filename, myList):
  with open(filename, 'w', newline='') as file:
    writer = csv.writer(file, delimiter='|')
    writer.writerows(myList)
    file.close()

def iterateThroughAllTags():
  with open(yamlFile) as f:
    data = yaml.load(f, Loader=SafeLoader)
    for keys,value in data['way-keys'].items():
      new_data = data['way-keys'][keys]['tags'].items()
      for k,v in new_data:
          for i,j in v.items():
            print(j)
            
#Function to create the quest, in particular it lets you add an element to the list
def createQuest(answers,node_or_way, question, id, key, key_value, tag, tag_value, verified, mylist, score, tag_answer, validated, icon):
  value = [node_or_way, question, id, key, key_value, tag, tag_value, verified, score,tag_answer, validated,answers, icon];
  mylist = initList(mylist, value)
  return mylist;

#Check wheter or not the element ahve value in the must_not_have_tags field inside the yaml file
def checkMustNotHaveTags(must_not, list_element):
  for k,v in must_not.items():
    if v in list_element['tags'] and v is not None:
      return False
  return True

def checkMustHaveValues(must_data, list_element):
  #Get data from the yaml file
  #for keys,value in data['way-keys'].items():
      new_data = must_data
      #Itero per ottenere i dati del campo muyst_have
      for k,v in new_data.items():
        tag_values_have = new_data[k]['values_to_have']
        tag_name = new_data[k]['name']
        tag_discards = new_data[k]['values_to_discard']
        #Check that the tag in the must_have_tab field is also in the list element
        #if it is then continue, else return false
        if tag_name not in list_element['tags'] and tag_name is not None:
          return False
        #if tag_discard is not None:
        #check that for every element inside the field "values_to_have" isn't None, if it is None then continue, if it isn't then make some other checks.
        checkNumber=0
        for i,j in tag_values_have.items():
          if j is not None:
            #Check if the value is the same as one of the values_to_have field
            if j == list_element['tags'][tag_name]:
              checkNumber = 1 #this means that I have an element that is equal, so I can go on
          else:
            checkNumber = 1
        if checkNumber == 0:
          return False
        #Check the tag_discards field, if even only one of this is the same as the element of the list then discard the element
        for i,j in tag_discards.items():
          if j is not None:
            #print("NOT NONE")
            if j == list_element['tags'][tag_name]:
              return False
      return True

#Function to check if the element is already in the list
def alreadyPresent(question_list, question, id):
  for element in question_list:
    if element[1] == question and id==element[2]:
      #print("ALREADY")
      #print(question)
      #if(element[0]=="NODE"):
      #  print(id + " " + question)
      return True
  return False

#Function used to check if the element contains the key I'm considering
def checkHasMyKey(my_key,i):
  if my_key in i['tags']:
    return True
  return False

#Function to check if the element doesn't contain certain values in the yaml file
def checkValuesNot(my_key,i,values_not_to_have):
  for k,v in values_not_to_have.items():
    #print(v)
    if str(i['tags'][my_key]) == v:
      return False
  return True

def checkValuesYes(my_key,i,values_to_have):
  for k,v in values_to_have.items():
    if v == None:
      return True
    if str(i['tags'][my_key]) == v:
      return True
  return False;

In [ ]:
#Create the single files for every node and way mission
import os

#question_list_sorted = question_list_sorted[:-1]

def writeInFile(element_to_write, path):
    with open(path, "a") as f:
        f.write(element_to_write)
        f.close()

def AddQuestions(my_list, my_node_path, my_way_path):
    for i in my_list:
        my_id = i[2]
        node_or_way = i[0]
        #answers = i[11]
        if node_or_way == "NODE":
            #print("chimichanga")
            path = my_node_path + "/" + str(my_id) + ".csv"
            my_str = ''.join(str(e) + "|" for e in i)
            writeInFile(my_str,path)
            writeInFile("\n", path)
        else:
            path = my_way_path + "/" +str(my_id) + ".csv"
            my_str = ''.join(str(e) + "|" for e in i)
            writeInFile(my_str,path)
            writeInFile("\n", path)

for i in cityFilesOrder:
    nodes_path = "tmp/nodes_question/" + i
    ways_path = "tmp/ways_question/" +i
    try:
        #os.mkdir("tmp/GeojsonFiles/centerWayPoints/" + i)
        os.mkdir("tmp/GeojsonFiles/singleWaysFiles/" + i)
        os.mkdir("tmp/GeojsonFiles/singleNodesFiles/" + i)
        os.mkdir("tmp/GeojsonFiles/Overpass/" + i)
        os.mkdir("tmp/XmlFiles/" + i)
        os.mkdir("tmp/allQuestions/" + i)
        os.mkdir(nodes_path)
        os.mkdir(ways_path)
    except OSError:
        print("creation failed")
    else:
        print("folders created")

In [ ]:
import csv

question_list = [["NODE OR WAY", "QUESTION", "ID", "KEY", "KEY_VALUE", "TAG", "TAG_VALUE", "VERIFIED","SCORE","TAGANSWER", "VALIDATING","ANSWERS","ICON"]]
#The tag answer represents the tag or the key that we have to search in the answer

#Start with the quest about the "WAY"

#Open the yaml file to get all the tag that I need
#cycle the way list and check the condition of the yaml file in order to initialize the questions.
#For every way inside the list verify if I need to create a question or not.

def createQuestList(waylistname, nodelistname, question_list):
  yamlFile="tmp/tagAndKeys_NewVersion_Answer.yaml"
  with open(yamlFile) as f:
      data = yaml.load(f, Loader=SafeLoader)
      for i in waylistname:
        for keys,value in data['way-keys'].items():
          new_data = data['way-keys'][keys]['tags']
          values_not_to_have = data['way-keys'][keys]['values_not_to_have']
          values_to_have = data["way-keys"][keys]["values"]
          questions_data = data['way-keys'][keys]['questions']
          must_data = data['way-keys'][keys]['must_have_tag']
          must_not_data = data['way-keys'][keys]['must_not_have_tag']
          my_key = data["way-keys"][keys]["name"]
          #Check that it contains my key
          if checkHasMyKey(my_key,i):
            #Check that my key doesn't have values contained in the values_not_to_have field inside the yaml file
            if checkValuesNot(my_key,i,values_not_to_have):
              #Check that my key contains the value inside the values section, if it does then go on. If there aren't value to check then continue
              if checkValuesYes(my_key,i,values_to_have):
                #check if the element has the must_have_tag
                if checkMustHaveValues(must_data, i) and checkMustNotHaveTags(must_not_data, i):
                #cycle through all the various tags in the tags section of the yaml file in order to create the questions
                  for k,v in new_data.items():
                    tag_value = new_data[k]['value']
                    tag_name = new_data[k]['name'] #THIS IS ALSO THE TAG THAT I HAVE TO SEARCH IN THE FUTURE.
                    tag_question = new_data[k]['question']
                    tag_score = new_data[k]['score']
                    validate_bool = new_data[k]['validating']
                    #print(new_data[k])
                    #possible_answers = ""
                    possible_answers = new_data[k]['answers']
                    icon = new_data[k]["icon"]
                    #If the value is none then I can instantly generate the question
                    if tag_value == 'None':
                      #Check that in b.ways there's not the attribute tag_name, if there isn't then I can create the question
                      if tag_name not in i['tags']:
                        #Check if the question is already in the list
                        if alreadyPresent(question_list, tag_question, str(i['id'])) == False: 
                          question_list = createQuest(possible_answers,'WAY', tag_question, str(i['id']), my_key, str(i['tags'][my_key]), tag_name, 'NONE', 'NO', question_list, tag_score, tag_name, validate_bool, icon)
                    else:
                      #Check whether or not the tag and the value inside the yaml file are also in the element, if they are then add the element to the questions
                      if tag_name in i['tags'] and tag_value == str(i['tags'][tag_name]):
                        #Check if the questions is already contained in the question list
                        if alreadyPresent(question_list, tag_question, str(i['id'])) == False: 
                          question_list = createQuest(possible_answers,'WAY', tag_question, str(i['id']), my_key, str(i['tags'][my_key]), tag_name, str(i['tags'][tag_name]), 'NO', question_list, tag_score, tag_name, validate_bool,icon)
                #Now I can add the question that are in the questions section of the yamls file. This questions do not need any verification process
                  for k,v in questions_data.items():
                    if v is not None:
                      #print(v)
                      question_value = questions_data[k]['question']
                      score_value = questions_data[k]['score']
                      answer_value = questions_data[k]['tagAnswer']
                      question_answers = questions_data[k]['answers']
                      icon = questions_data[k]["icon"]
                      #print(question_value)
                      #print(score_value)
                      if question_value is not None:
                        if alreadyPresent(question_list, question_value, str(i['id'])) == False:
                          question_list = createQuest(question_answers,"WAY", question_value, str(i['id']), my_key, str(i['tags'][my_key]), 'NONE', 'NONE', 'NO', question_list, score_value, answer_value,"no",icon)

  #Same thing for the node questions
  with open(yamlFile) as f:
      data = yaml.load(f, Loader=SafeLoader)
      for i in nodelistname:
        for keys,value in data['node-keys'].items():
          new_data = data['node-keys'][keys]['tags']
          #print(new_data)
          questions_data = data['node-keys'][keys]['questions']
          my_key = data["node-keys"][keys]["name"]
          my_values = data["node-keys"][keys]["values"]
          if checkHasMyKey(my_key,i):
            #Check the values field, my key needs to have the value written in the name field and then I create the mission associated
            for k,v in my_values.items():
              value_name = my_values[k]['name']
              value_question = my_values[k]['question']
              value_score = my_values[k]['score']
              value_answer = my_values[k]['tagAnswers']
              value_validating = my_values[k]['validating']
              possible_answers = my_values[k]['answers']
              icon = my_values[k]['icon']
              #print(possible_answers)
              if value_name is not None:
                #Create the mission if it contains the value
                if value_name == i['tags'][my_key]:
                  if alreadyPresent(question_list, value_question, str(i['id'])) == False: 
                    question_list = createQuest(possible_answers,"NODE", value_question, str(i['id']), my_key, str(i['tags'][my_key]),'NONE', 'NONE','NO',question_list, value_score, value_answer, value_validating,icon)

            #Cycle through all the various tags in the tags section of the yaml file in order to create the questions
            for k,v in new_data.items():
              tag_value = new_data[k]['value']
              tag_name = new_data[k]['name']
              tag_question = new_data[k]['question']
              tag_score = new_data[k]['score']
              validating_value = new_data[k]['validating']
              #print(new_data[k])
              tag_answers = new_data[k]['answers']
              icon = new_data[k]["icon"]
              #tag_answer = new_data["tagAnswers"]
              #If the value is None then you can create the question
              if tag_value == 'None':
                #Check that there's no attribute tag_name in the b.ways, if there isn't then create the question
                if tag_name not in i['tags']:
                  #Check that the questions isn't already in the list
                  if alreadyPresent(question_list, tag_question, str(i['id'])) == False: 
                    question_list = createQuest(tag_answers ,'WAY', tag_question, str(i['id']), my_key, str(i['tags'][my_key]), tag_name, 'NONE', 'NO', question_list, tag_score, tag_name, validating_value,icon)
              else:
                #Check if the tag and the value in the yaml file are also in the element in the list, if they are then add the element to the questions
                if tag_name in i['tags'] and tag_value == str(i['tags'][tag_name]):
                  #Check if the way already contains the question I'm creating
                  if alreadyPresent(question_list, tag_question, str(i['id'])) == False: 
                    question_list = createQuest(tag_answers ,'WAY', tag_question, str(i['id']), my_key, str(i['tags'][my_key]), tag_name, str(i['tags'][tag_name]), 'NO', question_list, tag_score, tag_name,validating_value,icon)
          
            #Now I can add the question that are in the questions section of the yaml file, these don't need any verification
            for k,v in questions_data.items():
              if v is not None:
                question_value = questions_data[k]['question']
                score_value = questions_data[k]['score']
                question_answers = questions_data[k]['answers']
                icon = questions_data[k]["icon"]
                if question_value is not None:
                  if alreadyPresent(question_list, question_value, str(i['id'])) == False:
                    question_list = createQuest(question_answers,"WAY", question_value, str(i['id']), my_key, str(i['tags'][my_key]), 'NONE', 'NONE', 'NO', question_list, score_value, my_key,"no",icon)
  return question_list            

j=0
for i in allWayKeyNames:
  nodes_path = "tmp/nodes_question/" + cityFilesOrder[j]
  ways_path = "tmp/ways_question/" + cityFilesOrder[j]
  question_list = [["NODE OR WAY", "QUESTION", "ID", "KEY", "KEY_VALUE", "TAG", "TAG_VALUE", "VERIFIED","SCORE","TAGANSWER", "VALIDATING","ANSWERS","ICON"]]
  question_list = createQuestList(i,allNodeKeyNames[j],question_list)
  filename = "tmp/allQuestions/" + cityFilesOrder[j] + "/wayQuests.csv"
  not_sorted = "tmp/allQuestions/" + cityFilesOrder[j] + "/wayQuestsNotSorted.csv"
  question_list_sorted = sorted(question_list, key=lambda x:x[2])
  question_list_sorted = question_list_sorted[:-1]
  createFile(filename, question_list_sorted)
  createFile(not_sorted, question_list)
  #AddQuestions(question_list_sorted, nodes_path, ways_path)
  j=j+1

In [ ]:
!pip install overpy
!pip install geojson
!pip install overpass

In [ ]:
#Function used to create the query for overpass
def createQuery(fileName, node_or_way, node_or_way_title):
  myQuery = node_or_way + "(id:"
  with open(fileName) as csv_file:
    csv_reader = csv.reader(csv_file, delimiter='|')
    line_count = 0
    for row in csv_reader:
      if line_count == 0:
        print(f'Column names are {", ".join(row)}')
        line_count += 1
      elif line_count ==1:
        if str(f'{row[0]}') == node_or_way_title:
          myQuery = myQuery + str(f'{row[2]}');
          line_count += 1
      else:
        if str(f'{row[0]}') == node_or_way_title:
          myQuery = myQuery + "," + str(f'{row[2]}');
          line_count += 1
    print(f'Processed {line_count} lines.')
  #myQuery = "[out:json];" + myQuery + ");(._;>;);out geom;"
  myQuery = myQuery + ");(._;>;);out meta;"
  return myQuery

#Function to create the quyery for overpy
def createQueryOverpy(fileName, node_or_way, node_or_way_title):
  myQuery = node_or_way + "(id:"
  with open(fileName) as csv_file:
    csv_reader = csv.reader(csv_file, delimiter='|')
    line_count = 0
    for row in csv_reader:
      if line_count == 0:
        print(f'Column names are {", ".join(row)}')
        line_count += 1
      elif line_count ==1:
        if str(f'{row[0]}') == node_or_way_title:
          myQuery = myQuery + str(f'{row[2]}');
          line_count += 1
      else:
        if str(f'{row[0]}') == node_or_way_title:
          myQuery = myQuery + "," + str(f'{row[2]}');
          line_count += 1
    print(f'Processed {line_count} lines.')
  myQuery = "[out:json];" + myQuery + ");(._;>;);out meta;"
  return myQuery


In [ ]:
!pip install geopandas

In [ ]:
#if this step gives errors it's probably because we have to wait some time between one call and the other. If that's the case then add a counter or something to do so.
import requests
import overpy
import geopandas as gpd

xml = []
nodeXml = []
nodeResult = []
api = overpy.Overpass()
url = "https://overpass-api.de/api/interpreter?"
#make the calls to overpass in order to extract the elements that have questions. 
for i in cityFilesOrder:
    file_to_use = "tmp/allQuestions/" + i + "/wayQuestsNotSorted.csv"
    myQuery = createQueryOverpy(file_to_use, "way", "WAY")
    myNodeQuery = createQueryOverpy(file_to_use, "node", "NODE")
    s = myQuery
    p = {'data':s}
    myNodeBody = {'data':myNodeQuery}
    r = requests.post(url,data=p)
    nodeResult = requests.post(url, data = myNodeBody)
    xml.append(r.text)
    nodeXml.append(nodeResult.text)

In [ ]:
#save the answer to some xml files
j=0
for i in cityFilesOrder:
  with open("tmp/XmlFiles/" + i + "/myXml.xml", "w") as f:
    f.write(xml[j])
    f.close()
  with open("tmp/XmlFiles/" + i + "/myNodeXml.xml", "w") as f:
    f.write(nodeXml[j])
    f.close()
  j=j+1

In [ ]:
pip install osm2geojson

In [ ]:
import osm2geojson
#convert the files in geojsons and shapes

g = []
nodeGeo = []
for i in cityFilesOrder:
    file_to_use = "tmp/allQuestions/" + i + "/wayQuestsNotSorted.csv"
    myQuery = createQueryOverpy(file_to_use, "way", "WAY")
    myNodeQuery = createQueryOverpy(file_to_use, "node", "NODE")
    s = myQuery
    g.append(osm2geojson.overpass_call(s))
    nodeGeo.append(osm2geojson.overpass_call(myNodeQuery))

In [ ]:
j=0
for i in cityFilesOrder:
  with open("tmp/GeojsonFiles/Overpass/" + i + "/myOverpass_call.geojson", "w") as f:
    f.write(g[j])
    f.close()
    j=j+1

j=0
for i in cityFilesOrder:
  with open("tmp/GeojsonFiles/Overpass/" + i + "myNodeCall.geojson", "w") as f:
    f.write(nodeGeo[j])
    f.close()
    j=j+1

In [ ]:
j=0
geojson_output = []
geojson_node_output = []
for i in cityFilesOrder:
    geojson_output.append(osm2geojson.json2shapes(g[j]))
    geojson_node_output.append(osm2geojson.json2shapes(nodeGeo[j]))
    type(geojson_output)
    j=j+1

In [ ]:
geojson_output[0][0]
#len(geojson_output)

In [ ]:
geojson_node_output[0][0]

In [ ]:
#Now I want to create a geojson for each single way and node.
import json


def createGeoJson(outputName, nodeOrWay, folder):
    for i in outputName:
        my_item = osm2geojson.shape_to_feature(i['shape'], i['properties'])
        with open(folder + nodeOrWay + str(i['properties']['id']) + ".geojson","w") as f:
            json.dump(my_item, f)
j=0
for i in cityFilesOrder:
    createGeoJson(geojson_output[j], "way", "tmp/GeojsonFiles/singleWaysFiles/" + i + "/")
    createGeoJson(geojson_node_output[j], "node", "tmp/GeojsonFiles/singleNodesFiles/" + i + "/")
    j=j+1

In [ ]:
geojson_output[0][0]['properties']

In [ ]:
#type(my_object)
#create the geodataframes
tutto = []
node_all = []
j=0
for i in cityFilesOrder:
    tutto.append(gpd.GeoDataFrame(geojson_output[j]))
    node_all.append(gpd.GeoDataFrame(geojson_node_output[j]))
    j=j+1

In [ ]:
node_all[0].head(5)

In [ ]:
#tutto

#tutto[1].properties[0]

In [ ]:
j=0
for i in cityFilesOrder:
    newTutto = tutto[j]
    newTutto['way'] = newTutto.properties.apply(lambda x: x['type'])
    newTutto['id'] = newTutto.properties.apply(lambda x: x['id'])
    tutto[j] = newTutto
    newNode = node_all[j]
    newNode['node'] = newNode.properties.apply(lambda x: x['type'])
    newNode['id'] = newNode.properties.apply(lambda x: x['id'])
    node_all[j] = newNode
    j = j+1

In [ ]:
#for elements in tutto.properties:
  #print(elements)
  #print(elements['tags'])
  #Uso try except perchè ci sono dei valori che potrebbero non avere la sezione tags.
  #try:
    #for tag in elements['tags']:
      #print(tag)
      #tutto[tag] = tutto.properties.apply(lambda x: x['tags'])

  #except Exception as e:
    #print(e)

In [ ]:
labels = []
node_labels = []

j=0
for i in cityFilesOrder:
  toIter = tutto[j]
  for index,row in toIter.iterrows():
    #print(index)
    try:
      node_key = row['properties']['tags'].keys()
      chiave = row['properties']['tags'].keys() 
      for c in chiave:
        if c not in labels:
          labels.append(c)
      for k in node_key:
        if k not in node_labels:
          node_labels.append(k)
    except KeyError as e:
      print(e) 
  j = j+1

In [ ]:
def getValue(row, label):
  v = ""
  #print(row)
  #row = row['tags']
  try:
    row = row['tags']
    v = row[label]
  except KeyError as e:
    #print(e)
    v = ""
    pass;
  return v

In [ ]:
j=0
for i in cityFilesOrder:
  tuttoNew = tutto[j]
  nodeNew = node_all[j]
  for label in labels:
    tuttoNew[label] = tuttoNew.properties.apply(lambda x: getValue(x,label))
    tutto[j] = tuttoNew
  for label in node_labels:
    nodeNew[label] = nodeNew.properties.apply(lambda x: getValue(x,label))
    node_all[j] = nodeNew
  j=j+1

In [ ]:
j=0
for i in cityFilesOrder:
    tuttoNew = tutto[j]
    tuttoNew['id'] = tuttoNew.properties.apply(lambda x: x['id'])
    tuttoNew['type'] = tuttoNew.properties.apply(lambda x: x['type'])
    tutto[j] = tuttoNew
    nodeNew = node_all[j]
    nodeNew['id'] = nodeNew.properties.apply(lambda x: x['id'])
    nodeNew['type'] = nodeNew.properties.apply(lambda x: x['type'])
    node_all[j] = nodeNew
    j=j+1

In [ ]:
print(tutto[0].head(3))
tutto[0]['shape'].head(1)

In [ ]:
node_all[0].head(5)

In [ ]:
print(type(tutto[0]['shape'].iloc[0]))

#this is necessary to find all the centroids
print(tutto[0]['shape'].iloc[0].representative_point())
gdf_center = []
j=0
for i in cityFilesOrder:
    gdf_c = tutto[j].copy()
    my_center_points = []
    for k in tutto[j]['shape']:
        centerPoint = k.representative_point()
        my_center_points.append(centerPoint)
    gdf_c = gdf_c.drop(['shape'], axis = 1)
    gdf_c['geometry'] = my_center_points #DATAFRAME WITH THE CENTERPOINT, I CAN REMOVE THE SHAPE SINCE I DON'T USE IT ANYMORE
    gdf_c.drop(gdf_c.columns.difference(['id','type','geometry','name', 'highway']),1,inplace=True)
    gdf_center.append(gdf_c)
    j=j+1
#gdf_center.head(1)
#tutto.head(1)

In [ ]:
j=0
for i in cityFilesOrder:
    tutto[j].drop(tutto[j].columns.difference(['shape','type','id']), 1, inplace=True)
    node_all[j].drop(node_all[j].columns.difference(['shape','type','id']), 1, inplace=True)
    j=j+1

In [ ]:
print(tutto[0].head(3))
node_all[0].head(3)

In [ ]:
print(tutto[0].shape)
print(node_all[0].shape)

In [ ]:
#create the geojson with all the centerpoints
j=0
for i in cityFilesOrder:
    my_centerJson = gdf_center[j].to_json()
    my_folder = "tmp/GeojsonFiles/centerWayPoints/"
    with open(my_folder + i + ".geojson","w") as f:
        f.write(my_centerJson)
        f.write("\n")
        #json.dump(my_centerJson, f)
    j=j+1

In [ ]:
import os
import sqlite3

DB_PATH = os.path.join(os.getcwd(), 'database_prova/withvalidation.db')

In [ ]:
#Create the table with the questions, but before doing so check for existing duplicates
import pandas as pd
if(os.path.exists('./database_prova/withvalidation.db')):
    databaseAlreadyExists = True
else:
    databaseAlreadyExists = False


for i in cityFilesOrder:
    my_data = pd.read_csv("./tmp/allQuestions/" + i + "/wayQuestsNotSorted.csv", index_col=False, delimiter='|')
    my_data = my_data.rename(columns={'NODE OR WAY':'TYPE'})#########
    my_data.drop(my_data.columns.difference(['TYPE','QUESTION','ID','SCORE','VALIDATING','ANSWERS','ICON', 'TAGANSWER']), 1, inplace=True)
    my_data["ANSWER"] = ""
    my_data["USERANSWERED"] = ""
    my_data["NUMBEROFVALIDATIONS"] = 0
    my_data["USERSWHOVALIDATED"] = ""
    print(my_data.head())

    with sqlite3.connect(DB_PATH) as conn: 
        df_sql = my_data.to_sql('question_table_tmp', conn, if_exists='append', index = False)

if(databaseAlreadyExists):
    query= """
        SELECT TYPE, QUESTION, ID FROM question_table_tmp 
        EXCEPT 
        SELECT TYPE, QUESTION, ID FROM question_table;
    """
else:
    query= """
        SELECT TYPE, QUESTION, ID, SCORE, VALIDATING, ANSWERS, ICON, TAGANSWER, ANSWER, USERANSWERED, NUMBEROFVALIDATIONS, USERSWHOVALIDATED  FROM question_table_tmp 
    """

with sqlite3.connect(DB_PATH) as conn: 
    new_entries = pd.read_sql(query, conn)

with sqlite3.connect(DB_PATH) as conn: 
    df_sql = new_entries.to_sql('question_table', conn, if_exists='append', index = False)


with sqlite3.connect(DB_PATH) as conn:
    conn.enable_load_extension(True)
    conn.load_extension("mod_spatialite")
    conn.execute("SELECT InitSpatialMetaData(1);")
    conn.execute("""SELECT ID FROM question_table ORDER BY ID""")
    conn.execute("""DROP TABLE IF EXISTS question_table_tmp """)

In [ ]:
#Get all the questions
query="""SELECT ID, TYPE FROM question_table""" #if you want to get the total score too then you can add the column SCORE there

with sqlite3.connect(DB_PATH) as conn: 
    idsScoresTypes = pd.read_sql(query, conn)


#create dataframe holding the sum of all the scores
sumDF = pd.DataFrame(idsScoresTypes)

sumDF = sumDF.groupby(['ID', "TYPE"]).sum()

sumDF.reset_index(inplace=True)

sumDF["ID"].astype(int)

sumDF["completed"] = "no"
sumDF.rename(columns = {'ID':'id', "TYPE":"type"}, inplace = True)
sumDF['type'] = sumDF['type'].apply(str.lower)

#If the database already exists then I have to update it, otherwise just create a new one.
if(databaseAlreadyExists):
    #If all the questions about a way or node have 5 validations then they are completed, else set completed to false.
    query = """SELECT ID FROM question_table WHERE ID IN (select ID from question_table group by ID having NUMBEROFVALIDATIONS=5)"""
    with sqlite3.connect(DB_PATH) as conn:
        allCompletedDF = pd.read_sql(query, conn)
    
    allCompletedDF["completed"] = "yes"
    allCompletedDF.rename(columns = {'ID':'id'}, inplace = True)
    sumDF.loc[sumDF['id'].isin(allCompletedDF['id']), 'completed'] = allCompletedDF['completed']

    with sqlite3.connect(DB_PATH) as conn:
        df_completed = sumDF.to_sql('completed_table',conn,if_exists="replace", index = False)
else:
    with sqlite3.connect(DB_PATH) as conn:
        df_completed = sumDF.to_sql('completed_table',conn,if_exists="replace", index = False)
#print(allCompletedDF)

In [ ]:
'''#Crete the table with the questions
import pandas as pd

for i in cityFilesOrder:
    my_data = pd.read_csv("./tmp/allQuestions/" + i + "/wayQuestsNotSorted.csv", index_col=False, delimiter='|')
    my_data = my_data.rename(columns={'NODE OR WAY':'TYPE'})#########
    my_data.drop(my_data.columns.difference(['TYPE','QUESTION','ID','SCORE','VALIDATING','ANSWERS','ICON', 'TAGANSWER']), 1, inplace=True)
    my_data["ANSWER"] = ""
    my_data["USERANSWERED"] = ""
    my_data["NUMBEROFVALIDATIONS"] = 0
    my_data["USERSWHOVALIDATED"] = ""
    print(my_data.head())

    with sqlite3.connect(DB_PATH) as conn: 
        df_sql = my_data.to_sql('question_table', conn, if_exists='append', index = False)'''

In [ ]:
'''#Take all the points of ways and nodes and sum the ones that have the same id. Put the value in the completed_table
def getAllScores(i):
    previousId = ""
    allScores = []
    currentId = ""
    fileName = "tmp/allQuestions/" + i + "/wayQuests.csv"
    with open(fileName) as csv_file:
        csv_reader = csv.reader(csv_file, delimiter='|')
        line_count = 0
        for row in csv_reader:
            if line_count == 0:
                previousId = row[2]
                currentId = row[2]
                score = row[8]
                element = {
                    "id": currentId,
                    "score": score
                }
                allScores.append(element);
                line_count=line_count+1
            else:
                currentId = row[2]
                score = row[8]
                if(currentId==previousId):
                    #modify existing element in list, do not append
                    previousScore = allScores[len(allScores)-1]['score'];
                    newScore = float(score) + float(previousScore)
                    score_to_write = str(newScore)
                    allScores[len(allScores)-1]['score'] = score_to_write
                    line_count = line_count+1;
                else:
                    #append element with score
                    score = row[8]
                    previousId = currentId
                    element = {
                        "id": currentId,
                        "score": score
                    }
                    allScores.append(element)
                    line_count = line_count+1;

    print(f'Processed {line_count} lines.')

    point_df = pd.DataFrame(allScores);
    return point_df'''

In [ ]:
'''#Create the table to check whether or not the id has all the answer
j=0
for i in cityFilesOrder:
    point_df = getAllScores(i)
    completed_data = tutto[j]
    #print(tutto[j])
    completed_data = completed_data.drop(['shape'], axis = 1)
    completed_data["completed"] = "no"

    completed_node = node_all[j]
    completed_node = completed_node.drop(['shape'], axis=1)
    completed_node["completed"] = "no"
    #print(completed_node.head())
    #print(completed_data.head())
    completed_node.append(completed_data)
    #print(type(completed_data))
    completed_data = gpd.GeoDataFrame( pd.concat( [completed_node,completed_data]) )
    #print(len(completed_node))
    #print(len(completed_data))
    print(completed_data.head())
    point_df["id"]=point_df["id"].astype(int)
    completed_data = pd.merge(completed_data,point_df, on="id")
    with sqlite3.connect(DB_PATH) as conn:
        df_completed = completed_data.to_sql('completed_table',conn,if_exists="append", index = False)
    j=j+1'''

In [ ]:
with sqlite3.connect(DB_PATH) as conn:
    conn.enable_load_extension(True)
    conn.load_extension("mod_spatialite")
    conn.execute("SELECT InitSpatialMetaData(1);")

DA QUA IN POI è COM'ERA UN TEMPO PER USARE TUTTO ASSIEME A QGIS E POTER QUINDI VISUALIZZARE LA MAPPA ASSIEME A TUTTI I DATI.

In [ ]:
'''#creo la tabella senza però mettere la proprietà della geometria.
df_way = tutto.drop(['shape'], axis = 1)
df_nodes = node_all.drop(['shape'], axis = 1)
with sqlite3.connect(DB_PATH) as conn: 
    df_way.to_sql('my_table_name', conn, if_exists='replace', index = False)
    df_nodes.to_sql('my_node_table', conn, if_exists='replace', index = False)'''

In [ ]:
'''!pip install sqlalchemy'''

In [ ]:
'''def init_dictionary(file_name, file_path, dict_list):
    for i in file_name:
        question = ""
        #print(ways_questions_path + "/" + i)
        with open(file_path + "/" + i) as f:
            csv_reader = csv.reader(f, delimiter = '|')
            lines = list(csv_reader)
            for row in lines:
                question = question + str(row[1]) + "|"
        question = question[:-1]
        #print(question + i)
        id = i
        id_split = id.split(".",1)
        sub_id = id_split[0]
        my_dict ={
            "id" : sub_id,
            "question" : question
        }
        dict_list.append(my_dict)
    return dict_list'''

In [ ]:
'''import geopandas
import pandas as pd
# shapely method to convert geometry objects into their well-known-binary representation
import shapely.wkb
# sqlite/spatialite
from sqlalchemy import create_engine, event
from sqlite3 import dbapi2 as sqlite
#file operations
import os

nodes_questions_path = "tmp/nodes_question"
ways_questions_path = "tmp/ways_question"
ways_files = os.listdir(ways_questions_path)
nodes_files = os.listdir(nodes_questions_path)
dict_list = []
node_dict_list = []
dict_list = init_dictionary(ways_files, ways_questions_path, dict_list)
node_dict_list = init_dictionary(nodes_files, nodes_questions_path, node_dict_list)

#https://blog.softhints.com/python-create-table-mysql-from-dictionary/
###https://gis.stackexchange.com/questions/141818/insert-geopandas-geodataframe-into-spatialite-database

one_way_shapefile = tutto
one_way_shapefile = geopandas.GeoDataFrame(one_way_shapefile, geometry='shape')
one_way_shapefile.to_file('MyGeometries.shp', driver='ESRI Shapefile')
gdf = geopandas.GeoDataFrame.from_file('MyGeometries.shp')
gdf= gdf.astype({"id": str}, errors='raise') 
file = "/MyGeometries.shp"
one_node_shapefile = node_all
one_node_shapefile = geopandas.GeoDataFrame(one_node_shapefile, geometry='shape')
one_node_shapefile.to_file('MyNodeGeometries.shp', driver='ESRI Shapefile')
node_gdf = geopandas.GeoDataFrame.from_file('MyNodeGeometries.shp')
node_gdf= node_gdf.astype({"id": str}, errors='raise') 
file = "/MyNodeGeometries.shp"


gdf['geometry'] = gdf.apply(lambda x: shapely.wkb.dumps(x.geometry), axis=1)
node_gdf['geometry'] = node_gdf.apply(lambda x: shapely.wkb.dumps(x.geometry), axis=1)
print('writing into database...')

with sqlite3.connect(DB_PATH) as conn: 
    gdf.to_sql('my_table_name', conn, if_exists='replace', index = False)
    node_gdf.to_sql('my_node_table', conn, if_exists='replace', index = False)

dict_df = pd.DataFrame.from_dict(dict_list)
node_dict_df = pd.DataFrame.from_dict(node_dict_list)
#dict_df = dict_df.astype({"id": str}, errors='raise') 

with sqlite3.connect(DB_PATH) as conn: 
    dict_df.to_sql('my_question_table', conn, if_exists='replace', index = False)
    node_dict_df.to_sql('my_node_question_table', conn, if_exists='replace', index = False)
#ELIMINO GLI ID CHE NON SONO PRESENTI DELLA TABELLA MY_TABLE_NAME
with sqlite3.connect(DB_PATH) as conn:
    conn.enable_load_extension(True)
    conn.load_extension("mod_spatialite")
    conn.execute("SELECT InitSpatialMetaData(1);")
    conn.execute("""DELETE 
                    FROM my_question_table
                    WHERE  ID NOT IN (SELECT id
                                FROM   my_table_name)""")
    conn.execute("""DELETE 
                    FROM my_node_question_table
                    WHERE  ID NOT IN (SELECT id
                                FROM   my_node_table)""")

with sqlite3.connect(DB_PATH) as conn:
    conn.enable_load_extension(True)
    conn.load_extension("mod_spatialite")
    conn.execute("SELECT InitSpatialMetaData(1);")
    conn.execute("""SELECT * FROM 'my_table_name' NATURAL JOIN 'my_question_table'""")
    conn.execute("""SELECT * FROM 'my_node_table' NATURAL JOIN 'my_node_question_table'""")
    conn.commit()

with sqlite3.connect(DB_PATH) as conn:
    conn.enable_load_extension(True)
    conn.load_extension("mod_spatialite")
    conn.execute("SELECT InitSpatialMetaData(1);")
    conn.execute("""DROP TABLE IF EXISTS merged""")
    conn.execute("""DROP TABLE IF EXISTS node_merged""")

with sqlite3.connect(DB_PATH) as conn:
    conn.enable_load_extension(True)
    conn.load_extension("mod_spatialite")
    conn.execute("SELECT InitSpatialMetaData(1);")
    conn.execute("""CREATE TABLE merged AS
                    SELECT * FROM "my_table_name"
                    NATURAL JOIN
                    'my_question_table'""")
    conn.execute("""CREATE TABLE node_merged AS
                    SELECT * FROM "my_node_table"
                    NATURAL JOIN
                    'my_node_question_table'""")

with sqlite3.connect(DB_PATH) as conn:
    conn.enable_load_extension(True)
    conn.load_extension("mod_spatialite")
    conn.execute("SELECT InitSpatialMetaData(1);")
    conn.execute(
        """SELECT AddGeometryColumn('merged', 'geom', 4326, 'LINESTRING', 2);"""
    )
    conn.execute(
        """SELECT AddGeometryColumn('node_merged', 'geom', 4326, 'POINT', 2);"""
    )

    conn.commit()

print("AFTER TABLE NAME")
with sqlite3.connect(DB_PATH) as conn:
    conn.enable_load_extension(True)
    conn.load_extension("mod_spatialite")
    conn.execute("UPDATE merged SET geom=GeomFromWKB(geometry, 4326);")
    conn.execute("UPDATE node_merged SET geom=GeomFromWKB(geometry, 4326);")
    conn.commit()
    
#engine.execute("SELECT AddGeometryColumn('my_table_name', 'geom', 4326, 'LINE', 2);")

#engine.execute("UPDATE my_table_name SET geom=GeomFromWKB(geometry, 4326);")
print("AFTER TABLE NAME")
#https://gis.stackexchange.com/questions/137149/how-to-delete-a-column-from-the-attribute-table-of-a-spatialite-layer
#connection = engine.connect()
with sqlite3.connect(DB_PATH) as conn:
    conn.execute("""BEGIN TRANSACTION""")
    conn.execute("""CREATE TEMPORARY TABLE t1_backup(id,type,question,geom)""")
    conn.execute("""INSERT INTO t1_backup SELECT id,type,question,geom FROM merged""")
    conn.execute("""DROP TABLE merged""")
    conn.execute("""CREATE TABLE merged(id,type,question,geom)""")
    conn.execute("""INSERT INTO merged SELECT id,type,question,geom FROM t1_backup""")
    conn.execute("""DROP TABLE t1_backup""")
    conn.commit();

with sqlite3.connect(DB_PATH) as conn:
    conn.execute("""BEGIN TRANSACTION""")
    conn.execute("""CREATE TEMPORARY TABLE tnode_backup(id,type,question,geom)""")
    conn.execute("""INSERT INTO tnode_backup SELECT id,type,question,geom FROM node_merged""")
    conn.execute("""DROP TABLE node_merged""")
    conn.execute("""CREATE TABLE node_merged(id,type,question,geom)""")
    conn.execute("""INSERT INTO node_merged SELECT id,type,question,geom FROM tnode_backup""")
    conn.execute("""DROP TABLE tnode_backup""")
    conn.commit();
# start functions
#writeIntoDatabase()'''

In [ ]:
'''#NOW I DROP THE TABLES THAT I DO NOT NEED ANYMORE
with sqlite3.connect(DB_PATH) as conn:
    conn.execute("""DROP TABLE IF EXISTS my_question_table """)
    conn.execute("""DROP TABLE IF EXISTS my_node_table """)
    conn.execute("""DROP TABLE IF EXISTS my_table_name """)
    conn.execute("""DROP TABLE IF EXISTS my_node_question_table""")
    conn.execute("""DROP TABLE IF EXISTS my_node_question_table""")
    conn.commit();
    #my_node_table
    #my_table_name'''

In [ ]:
'''#MOSTRA TUTTI GLI ID DELLE STRADE DISTANTI TOT METRI DAL MIO PUNTO
with sqlite3.connect(DB_PATH) as conn:
    conn.enable_load_extension(True)
    conn.load_extension("mod_spatialite")
    cur = conn.execute(
        """SELECT id from node_merged where ST_Distance(GeomFromText('POINT(11.194239 46.052415)',4326),geom,0)< 2500.120;"""
    )
    results = cur.fetchall()
print(results)
conn.close()'''

In [ ]:
'''#MOSTRA DISTANZA DA UN DATO PUNTO DI TUTTI I MIEI ELEMENTI
with sqlite3.connect(DB_PATH) as conn:
    conn.enable_load_extension(True)
    conn.load_extension("mod_spatialite")
    cur = conn.execute(
        """select ST_Distance(GeomFromText('POINT(11.194239 46.052415)',4326),geom) from merged;"""
    )
    results = cur.fetchall()
print(results)
conn.close()



#11.19314879999986,46.05232919995909
#11.19536759999801,46.05260819995721'''

In [ ]:
'''#with open("/tmp/myjsonfile.geojson","w") as f:
#  json.dump(my_object, f);

#lo 0 sta per "usa ellissi" https://www.gaia-gis.it/gaia-sins/spatialite-sql-4.3.0.html
with sqlite3.connect(DB_PATH) as conn:
    conn.enable_load_extension(True)
    conn.load_extension("mod_spatialite")
    cur = conn.execute(
        """SELECT id, AsText(geom) from merged where ST_Distance(GeomFromText('POINT(11.194239 46.052415)',4326),geom,0)< 111.120;"""
    )
    results = cur.fetchall()
print(results)
conn.close()

#il valore dopo il < è già espresso in metri


the unit returned by ST_Distance(), ST_Length() and ST_Area() exactly is the one defined by the corresponding SRID.
consequently, if you are using latitude and longitude (SRID=4326, WGS 84), any length will be measured in DEGREES, and any area in SQUARE DEGREES.
degree latitude is exactly 60 nautical miles apart. That's 111,120 meters or 364,567 feet. Just multiply the value from Distance() with 111,120 to get the distance in meters.



#select id, AsText(geom) from my_table_name where ST_Distance(GeomFromText('POINT(11.130962 46.012215)',4326),geom,0)< 20000.0;'''

In [ ]:
#https://gis.stackexchange.com/questions/353082/plotting-shapely-multipolygon-using-matplotlib

#fig, axs = plt.subplots()
#fig, ax = plt.subplots(figsize=(12,12))
#fig.set_size_inches(12, 12, forward=True)
#axs.set_aspect('equal', 'datalim')

#for geom in trento:    
#    xs, ys = geom.exterior.xy    
#    axs.fill(xs, ys, alpha=0.5, fc='r', ec='none')

#plt.show()
